#  Анализ покупок

Работа с чеками и корреляциями в покупках.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

Подружаем данные и смотрим как они выглядят.

In [4]:
df = pd.read_csv('Прикладная статистика\Данные\groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

(38765, 3)


,id,fielddate,product
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [5]:
product_name = df['product'].value_counts().index.values[0]
product_name

'whole milk'

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [6]:
n_cnt = df['id'].value_counts().size
n_cnt

3898

В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

In [8]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
id,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,1,0
1001,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,2,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0


__в)__ Постройте матрицу корреляций Пирсона.

In [9]:
sales_correlation = sparse_sales.corr()
sales_correlation.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
product,,,,,,,,,,,,,,,,,,,,,
Instant food products,1.000000,-0.006936,-0.009420,-0.010825,-0.003470,-0.004007,-0.010419,-0.008275,0.005847,-0.014528,...,0.025355,0.021851,0.005348,0.014025,-0.005670,0.016640,0.002853,0.015981,0.005071,0.018221
UHT-milk,-0.006936,1.000000,0.013806,0.006105,-0.007877,0.018349,0.009462,-0.018785,0.002897,0.020801,...,-0.003803,-0.024466,0.009144,0.006618,-0.012871,-0.011550,0.009990,0.028747,0.006505,0.028753
abrasive cleaner,-0.009420,0.013806,1.000000,-0.006523,-0.002091,-0.002415,-0.013429,-0.004986,0.018970,0.001813,...,0.012922,-0.008507,-0.007390,0.002163,-0.003417,0.010777,0.000107,0.003558,0.008360,-0.009420
artif. sweetener,-0.010825,0.006105,-0.006523,1.000000,0.105251,-0.002775,-0.015432,-0.005730,-0.006645,0.016042,...,0.049392,-0.009776,0.010179,-0.006614,-0.003926,-0.006806,-0.018394,0.029591,-0.024397,-0.010825
baby cosmetics,-0.003470,-0.007877,-0.002091,0.105251,1.000000,-0.000889,-0.004947,-0.001837,0.014798,-0.007983,...,-0.003919,-0.003134,-0.007413,-0.011288,-0.001259,-0.008476,-0.005896,-0.021056,-0.000410,-0.003470


Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [10]:
top = sales_correlation['domestic eggs'].sort_values(ascending = False)
top_1 = top.index.values[1]
top_1

'meat spreads'

Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [11]:
bottom = sales_correlation['domestic eggs'].sort_values(ascending = True)
bottom_1 = bottom.index.values[1]
bottom_1 

'cooking chocolate'

Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [12]:
unique_products = df['product'].unique()

product = np.random.choice(unique_products)
print(product)
sales_correlation[product].sort_values(ascending = False).head(10)
sales_correlation[product].sort_values(ascending = True).head(10)

baking powder


product
napkins                     -0.031884
semi-finished bread         -0.026665
photo/film                  -0.024999
packaged fruit/vegetables   -0.023886
sweet spreads               -0.023582
herbs                       -0.021168
vinegar                     -0.020127
spread cheese               -0.019290
house keeping products      -0.019263
cereals                     -0.018603
Name: baking powder, dtype: float64

__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [13]:
S = sales_correlation.values
S = S - np.eye(S.shape[0])
answer = sales_correlation.iloc[np.where(S == S.max())].index
answer

Index(['preservation products', 'soups'], dtype='object', name='product')

## 2. Зависимость. 

Cобытия $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

In [14]:
ab = (sparse_sales [['whole milk', 'domestic eggs']] > 1).all(axis=1).mean()
a = (sparse_sales ['whole milk'] > 1).mean()
b = (sparse_sales ['domestic eggs'] > 1).mean()
answer = ab/(a*b)
answer

1.9908069458631261

__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавим к знаменателю маленькое число, например `1e-10`.

In [15]:
from itertools import combinations
from tqdm.notebook import tqdm
food_lift = dict()

for item,jtem in tqdm(combinations(unique_products, 2)):
    ab = (sparse_sales [[item, jtem]] > 1).all(axis=1).mean()
    a = (sparse_sales [item] > 1).mean()
    b = (sparse_sales [jtem] > 1).mean()
    food_lift[(item,jtem)] = ab/(a*b + 1e-10)

0it [00:00, ?it/s]

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [16]:
food = [item for item in sorted(food_lift.items(), key=lambda w: w[1], reverse=True) if item[1] > 0]
answer = len(food)
answer

723

Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [19]:
food[:10]

[(('pasta', 'pudding powder'), 1298.6755799524421),
 (('misc. beverages', 'cling film/bags'), 487.1574741565755),
 (('candy', 'semi-finished bread'), 433.0380027293242),
 (('meat', 'photo/film'), 324.7922081331131),
 (('hamburger meat', 'popcorn'), 278.3983564492798),
 (('frozen meals', 'frozen dessert'), 278.3983564492798),
 (('oil', 'instant coffee'), 243.6018663426727),
 (('turkey', 'long life bakery product'), 149.91431592814772),
 (('cream cheese ', 'frozen dessert'), 149.91431592814772),
 (('oil', 'seasonal products'), 121.80671631107019)]

Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [20]:

ab = (sparse_sales [['pasta', 'pudding powder']] > 1).all(axis=1).sum()
a = (sparse_sales ['pasta'] > 1).sum()
b = (sparse_sales ['pudding powder'] > 1).sum()
ab, a, b

(1, 3, 1)

Для какой пары продуктов метрика оказывается самой маленькой? 

In [21]:
food[-10:]

[(('butter', 'root vegetables'), 0.6584457769477846),
 (('soda', 'frozen vegetables'), 0.6187300095042264),
 (('other vegetables', 'dessert'), 0.6065980514021808),
 (('coffee', 'soda'), 0.5728981671833303),
 (('root vegetables', 'pork'), 0.5725615643400158),
 (('chicken', 'soda'), 0.5333879569716461),
 (('yogurt', 'brown bread'), 0.5233618696002728),
 (('root vegetables', 'bottled beer'), 0.4180608499336609),
 (('soda', 'curd'), 0.368291708276899),
 (('whole milk', 'coffee'), 0.2703564703294116)]

## 3. Расчет метрик только для часто покупаемых товаров
 

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. 

In [23]:
food_lift = dict()

for item,jtem in tqdm(combinations(unique_products, 2)):
    ab = (sparse_sales [[item, jtem]] > 1).all(axis=1).sum()
    a = (sparse_sales [item] > 1).sum()
    b = (sparse_sales [jtem] > 1).sum()
    
    if (a > 10)&(b > 10):
        food_lift[(item,jtem)] = ab/(a*b + 1e-10)
    
food = [item for item in sorted(food_lift.items(), key=lambda w: w[1], reverse=True) if item[1] > 0]
len(food)
food[:10]
food[-10:]

[(('butter', 'root vegetables'), 0.00016891891891891606),
 (('soda', 'frozen vegetables'), 0.0001587301587301562),
 (('other vegetables', 'dessert'), 0.00015561780267662377),
 (('coffee', 'soda'), 0.00014697236919458925),
 (('root vegetables', 'pork'), 0.0001468860164512317),
 (('chicken', 'soda'), 0.0001368363437328936),
 (('yogurt', 'brown bread'), 0.00013426423200859112),
 (('root vegetables', 'bottled beer'), 0.0001072501072501061),
 (('soda', 'curd'), 9.44822373393793e-05),
 (('whole milk', 'coffee'), 6.93577472603685e-05)]